In [1]:
# 유저 기반 협업 필터링을 통한 상품 추천 시스템
import pandas as pd
import numpy as np
from math import sqrt

In [2]:
#평점 데이터
RATING_DATA_PATH = '../data/ratings.csv'
np.set_printoptions(precision=2)

In [3]:
# 유저 간의 유클리드 거리를 계산해주는 함수
def distance(user_1, user_2):
    return sqrt(np.sum((user_1 - user_2)**2))

In [4]:
# 특정 영화를 평가하지 않은 유저들을 제외시켜주는 함수
def filter_users_without_movie(rating_data, movie_id):
    return rating_data[~np.isnan(rating_data[:,movie_id])]

In [5]:
# 평점 데이터의 빈값들을 각 유저 평균 값으로 채워주는 함수
def fill_nan_with_user_mean(rating_data):
    filled_data = np.copy(rating_data)  # 평점 데이터를 훼손하지 않기 위해 복사
    row_mean = np.nanmean(filled_data, axis=0)  # 유저 평균 평점 계산
    
    inds = np.where(np.isnan(filled_data))  # 비어 있는 인덱스들을 구한다
    filled_data[inds] = np.take(row_mean, inds[1])  #빈 인덱스를 유저 평점으로 채운다
    
    return filled_data

In [6]:
# 유저의 이웃들을 찾아주는 함수
def get_k_neighbors(user_id, rating_data, k):
    distance_data = np.copy(rating_data)  # 평점 데이터를 훼손하지 않기 위해 복사
    # 마지막에 거리 데이터를 담을 열 추가
    distance_data = np.append(distance_data, np.zeros((distance_data.shape[0], 1)), axis=1)
    
    for i in range(len(distance_data)):
        row = distance_data[i]
        if i == user_id:  # 같은 유저면 거리를 무한대로 설정
            row[-1] = np.inf
        else:  # 다른 유저면 마지막 열에 거리 데이터를 저장
            row[-1] = distance(distance_data[user_id][:-1], row[:-1])
    
    # 데이터를 거리 열을 기준으로 정렬한다
    distance_data = distance_data[np.argsort(distance_data[:, -1])]
    
    # 가장 가까운 k개의 행만 리턴. 마지막 열은 제외
    return distance_data[:k, :-1]

In [7]:
# 유저의 영화 평점 예측 값 구하기
def predict_user_rating(rating_data, k, user_id, movie_id,):
    # movie_id 번째 영화를 보지 않은 유저를 데이터에서 미리 제외
    filtered_data = filter_users_without_movie(rating_data, movie_id)
    # 빈값들이 채워진 새로운 행렬을 만든다
    filled_data = fill_nan_with_user_mean(filtered_data)
    # 유저 user_id와 비슷한 k개의 유저 데이터를 찾는다
    neighbors = get_k_neighbors(user_id, filled_data, k)
    
    return np.mean(neighbors[:, movie_id])

In [8]:
rating_data = pd.read_csv(RATING_DATA_PATH, index_col='user_id').values
# 5개의 이웃들을 써서 유저 0의 영화 3에 대한 예측 평점 구하기
predict_user_rating(rating_data, 5, 0, 3)  

4.0